In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
data = pd.read_csv(r"C:\Users\Trilokesh\Downloads\Datas\MIP.csv.csv")

In [3]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-02-05,262.000000,267.899994,250.029999,254.259995,254.259995,11896100
1,2018-02-06,247.699997,266.700012,245.000000,265.720001,265.720001,12595800
2,2018-02-07,266.579987,272.450012,264.329987,264.559998,264.559998,8981500
3,2018-02-08,267.079987,267.619995,250.000000,250.100006,250.100006,9306700
4,2018-02-09,253.850006,255.800003,236.110001,249.470001,249.470001,16906900
...,...,...,...,...,...,...,...
1004,2022-01-31,401.970001,427.700012,398.200012,427.140015,427.140015,20047500
1005,2022-02-01,432.959991,458.480011,425.540009,457.130005,457.130005,22542300
1006,2022-02-02,448.250000,451.980011,426.480011,429.480011,429.480011,14346000
1007,2022-02-03,421.440002,429.260010,404.279999,405.600006,405.600006,9905200


In [4]:
data.isnull()

,Date,Open,High,Low,Close,Adj Close,Volume
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
1004,False,False,False,False,False,False,False
1005,False,False,False,False,False,False,False
1006,False,False,False,False,False,False,False
1007,False,False,False,False,False,False,False


In [5]:
data.shape

(1009, 7)

In [6]:
data['Date'] = pd.to_datetime(data.Date)

In [7]:
print("Starting date: ",data.iloc[0][0])
print("Ending date: ", data.iloc[-1][0])
print("Duration: ", data.iloc[-1][0] - data.iloc[0][0])

Starting date:  2018-02-05 00:00:00
Ending date:  2022-02-04 00:00:00
Duration:  1460 days 00:00:00


In [8]:
closedf = data[['Date','Close']]
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (1009, 2)


In [9]:
close_stock = closedf.copy()
del closedf['Date']
scaler = MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

(1009, 1)


In [10]:
training_size = int(len(closedf)*0.65)
testing_size = len(closedf) - training_size
training_data, testing_data = closedf[0:training_size, :], closedf[training_size: len(closedf), :1]
print('Training_date: ', training_data.shape)
print("Testing_date: ", testing_data.shape)

Training_date:  (655, 1)
Testing_date:  (354, 1)


In [11]:
def creat_dateset(dataset, time_step = 1):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [12]:
time_step = 15
X_training, y_training = creat_dateset(training_data, time_step)
X_testing, y_testing = creat_dateset(testing_data, time_step)
print("X_training: ", X_training.shape)
print("y_training: ", y_training.shape)
print("X_testing: ", X_testing.shape)
print("y_testing", y_testing.shape)

X_training:  (639, 15)
y_training:  (639,)
X_testing:  (338, 15)
y_testing (338,)


In [13]:
X_training = X_training.reshape(X_training.shape[0], X_training.shape[1], 1)
X_testing = X_testing.reshape(X_testing.shape[0], X_testing.shape[1], 1)

print("X_Train: ", X_training.shape)
print("X_test: ", X_testing.shape)

X_Train:  (639, 15, 1)
X_test:  (338, 15, 1)


In [14]:
tf.keras.backend.clear_session()
model = Sequential()
model.add(LSTM(32, return_sequences = True,input_shape = (time_step, 1)))
model.add(LSTM(32,return_sequences = True))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 32)            4352      
                                                                 
 lstm_1 (LSTM)               (None, 15, 32)            8320      
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 21025 (82.13 KB)
Trainable params: 21025 (82.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
model.fit(X_training,y_training,validation_data=(X_testing,y_testing),epochs=200,batch_size=5,verbose=1)

Epoch 1/200
128/128 [==============================] - 2s 13ms/step - loss: 5.1638e-04 - val_loss: 0.0024
Epoch 2/200
128/128 [==============================] - 2s 14ms/step - loss: 4.6750e-04 - val_loss: 0.0022
Epoch 3/200
128/128 [==============================] - 2s 14ms/step - loss: 5.2207e-04 - val_loss: 0.0019
Epoch 4/200
128/128 [==============================] - 2s 12ms/step - loss: 4.9310e-04 - val_loss: 0.0019
Epoch 5/200
128/128 [==============================] - 2s 14ms/step - loss: 4.9734e-04 - val_loss: 0.0019
Epoch 6/200
128/128 [==============================] - 2s 13ms/step - loss: 4.6463e-04 - val_loss: 0.0029
Epoch 7/200
128/128 [==============================] - 2s 12ms/step - loss: 4.6670e-04 - val_loss: 0.0015
Epoch 8/200
128/128 [==============================] - 2s 14ms/step - loss: 5.2114e-04 - val_loss: 0.0024
Epoch 9/200
128/128 [==============================] - 2s 14ms/step - loss: 5.1109e-04 - val_loss: 0.0027
Epoch 10/200
128/128 [========================

128/128 [==============================] - 2s 18ms/step - loss: 4.6294e-04 - val_loss: 0.0039
Epoch 78/200
128/128 [==============================] - 2s 19ms/step - loss: 4.9641e-04 - val_loss: 0.0039
Epoch 79/200
128/128 [==============================] - 2s 15ms/step - loss: 4.6282e-04 - val_loss: 0.0036
Epoch 80/200
128/128 [==============================] - 2s 19ms/step - loss: 4.5285e-04 - val_loss: 0.0030
Epoch 81/200
128/128 [==============================] - 2s 15ms/step - loss: 5.1093e-04 - val_loss: 0.0054
Epoch 82/200
128/128 [==============================] - 2s 15ms/step - loss: 5.0433e-04 - val_loss: 0.0046
Epoch 83/200
128/128 [==============================] - 2s 13ms/step - loss: 4.5393e-04 - val_loss: 0.0063
Epoch 84/200
128/128 [==============================] - 2s 14ms/step - loss: 4.4637e-04 - val_loss: 0.0043
Epoch 85/200
128/128 [==============================] - 2s 18ms/step - loss: 4.4612e-04 - val_loss: 0.0041
Epoch 86/200
128/128 [============================

128/128 [==============================] - 2s 16ms/step - loss: 4.7855e-04 - val_loss: 0.0105
Epoch 154/200
128/128 [==============================] - 2s 18ms/step - loss: 4.5023e-04 - val_loss: 0.0089
Epoch 155/200
128/128 [==============================] - 3s 21ms/step - loss: 4.5070e-04 - val_loss: 0.0080
Epoch 156/200
128/128 [==============================] - 2s 15ms/step - loss: 4.8617e-04 - val_loss: 0.0108
Epoch 157/200
128/128 [==============================] - 2s 15ms/step - loss: 4.3819e-04 - val_loss: 0.0067
Epoch 158/200
128/128 [==============================] - 2s 13ms/step - loss: 4.4163e-04 - val_loss: 0.0085
Epoch 159/200
128/128 [==============================] - 3s 20ms/step - loss: 4.4596e-04 - val_loss: 0.0075
Epoch 160/200
128/128 [==============================] - 2s 18ms/step - loss: 4.5174e-04 - val_loss: 0.0088
Epoch 161/200
128/128 [==============================] - 2s 16ms/step - loss: 4.3330e-04 - val_loss: 0.0094
Epoch 162/200
128/128 [===================

In [41]:
training_predict = model.predict(X_training)
testing_predict = model.predict(X_testing)
training_predict.shape, testing_predict.shape

11/11 [==============================] - 0s 4ms/step


((639, 1), (338, 1))

In [42]:
training_predict = scaler.inverse_transform(training_predict)
testing_predict = scaler.inverse_transform(testing_predict)
original_ytraining = scaler.inverse_transform(y_training.reshape(-1, 1))
original_ytesting = scaler.inverse_transform(y_testing.reshape(-1,1))

In [43]:
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytraining,training_predict)))
print("Train data MSE: ", mean_squared_error(original_ytraining,training_predict))
print("Test data MAE: ", mean_absolute_error(original_ytraining,training_predict))
print("\n")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytesting,testing_predict)))
print("Test data MSE: ", mean_squared_error(original_ytesting,testing_predict))
print("Test data MAE: ", mean_absolute_error(original_ytesting,testing_predict))

Train data RMSE:  9.17723356247127
Train data MSE:  84.22161586014913
Test data MAE:  6.763264762887872


Test data RMSE:  48.44988641512451
Test data MSE:  2347.3914936384663
Test data MAE:  34.54716050079165


In [39]:
print("Train data explained variance regression score:", explained_variance_score(original_ytraining, training_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytesting, testing_predict))

Train data explained variance regression score: 0.9751983140135186
Test data explained variance regression score: 0.9131471545138883


In [21]:
print("Train data R2 score:", r2_score(original_ytraining, training_predict))
print("Test data R2 score:", r2_score(original_ytesting, testing_predict))

Train data R2 score: 0.970348346861059
Test data R2 score: 0.9166677550784493


In [22]:
print("Train data MGD: ", mean_gamma_deviance(original_ytraining, training_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytesting, testing_predict))
print("\n")
print("Train data MPD: ", mean_poisson_deviance(original_ytraining, training_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytesting, testing_predict))

Train data MGD:  0.0008240829727366655
Test data MGD:  0.0010395177815646474


Train data MPD:  0.2971546300588877
Test data MPD:  0.5449646367878238


In [29]:
look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(training_predict)+look_back, :] = training_predict
print("Train predicted data: ", trainPredictPlot.shape)

testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(training_predict)+(look_back*2)+1:len(closedf)-1, :] = testing_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'Date': close_stock['Date'],
                       'original_close': close_stock['Close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['Date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','Date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1009, 1)
Test predicted data:  (1009, 1)


In [31]:
x_input=testing_data[len(testing_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        
        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        
        lst_output.extend(yhat.tolist())
        i=i+1
               
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


In [32]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [33]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')

fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [34]:
lstmdf=closedf.tolist()
lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(lstmdf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')

fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()